This Notebook will mainly unsed for the final Capstone Project.

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


Read the Wikipedia URL-information

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

Use the HTML content to convert into pandas dataframe. "Not assigned" and "nan" is filtered out for the column "Borough".

In [3]:
df = dfs[0]
df.where(df["Borough"]!="Not assigned", inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Use the .shape method to print the number of rows of the dataframe

In [4]:
df.shape

(103, 3)

This is to import the csv-file with the langitute information

In [5]:
df_geo = pd.read_csv("https://cocl.us/Geospatial_data")
df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Now join with existing dataframe

In [6]:
df = df.merge(df_geo, on ='Postal Code', how='left')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


Now explore the data for better visualization

In [7]:
df.Borough.unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

Import all necessary libraries

In [9]:
pip install folium

     |████████████████████████████████| 93 kB 3.6 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Get location data of Toronto

In [11]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create Map of Toronto

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We see all the observations graphically throughout the map

Let's cluster the Neighbourhoods in the Borough "York"

In [13]:
york_data = df[df['Borough'].str.contains("York")].reset_index(drop=True)
york_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


Let's get the geographical coordinates of York.

In [15]:
address = 'York, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of York are 43.6896191, -79.479188.


Create Map of York

In [18]:
# create map of York using latitude and longitude values
map_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(york_data['Latitude'], york_data['Longitude'], york_data['Borough'], york_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

We notice that York is in the northern part of town

Now let's explore the area!

In [31]:
CLIENT_ID = 'OSEO0CTWLPBA5D4C3SUQ55EE1JLPPPAIYN3RYAECQUI10L3U' # your Foursquare ID
CLIENT_SECRET = 'ENUQSRXBH00V30AEHILA3QP5HLCXHRDH2HNVQFMG5HIAGHGX' # your Foursquare Secret
ACCESS_TOKEN = 'UNOFVKGGOI1JNSOCH43MV2LQHGFGUG4CBTED51J53A4M0UEA' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OSEO0CTWLPBA5D4C3SUQ55EE1JLPPPAIYN3RYAECQUI10L3U
CLIENT_SECRET:ENUQSRXBH00V30AEHILA3QP5HLCXHRDH2HNVQFMG5HIAGHGX


Assume we are looking for some Food on Wilson Avenue

In [54]:
address = '1 Wilson Avenue, York, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7320723 -79.4592056


Now let's look for some Chinese food in York

In [47]:
search_query = 'Chinese'
radius = 1000
print(search_query + ' .... OK!')

Chinese .... OK!


Define the corresponding URL

In [55]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OSEO0CTWLPBA5D4C3SUQ55EE1JLPPPAIYN3RYAECQUI10L3U&client_secret=ENUQSRXBH00V30AEHILA3QP5HLCXHRDH2HNVQFMG5HIAGHGX&ll=43.7320723,-79.4592056&oauth_token=UNOFVKGGOI1JNSOCH43MV2LQHGFGUG4CBTED51J53A4M0UEA&v=20180604&query=Chinese&radius=1000&limit=30'

Send the GET Request and examine the results

In [56]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc3ee676985792a449f6f65'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4db8b8211e7206c23d0ece7d',
    'name': 'Blossom Chinese Restaurant',
    'location': {'lat': 43.73082072410727,
     'lng': -79.46567250883152,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.73082072410727,
       'lng': -79.46567250883152}],
     'distance': 538,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1606676071',
    'hasPerk': False}]}}

Get relevant part of JSON and transform it into a pandas dataframe

In [57]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df_chinese = json_normalize(venues)
df_chinese.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4db8b8211e7206c23d0ece7d,Blossom Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1606676071,False,43.730821,-79.465673,"[{'label': 'display', 'lat': 43.73082072410727...",538,CA,Toronto,ON,Canada,"[Toronto ON, Canada]"


In [58]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_chinese.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df_chinese.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,id
0,Blossom Chinese Restaurant,Chinese Restaurant,43.730821,-79.465673,"[{'label': 'display', 'lat': 43.73082072410727...",538,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",4db8b8211e7206c23d0ece7d


In [60]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Wilson Avenue',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Chinese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

We see that there is indeed a Chinese Retaurant on Wilson Avenue.